In [1]:
import sys
sys.path.append("/home/aiuser/work/userlib/BigQuant_Resources_Collection/BigQuant_Resources_Collection/00_General_Resources/General_Tool")
from cn_general_tool import *

In [2]:
sql = """
WITH 
data_base AS (
    SELECT date
    FROM all_trading_days
    WHERE market_code = 'CN'
    QUALIFY date >= '1990-01-01'
)
SELECT 
    date,
    ROW_NUMBER(date) AS date_index,
FROM data_base
ORDER BY date
"""

df = dai.query(sql).df()

In [3]:
df["dt_year"]      = df["date"].dt.year.astype(str)
df["dt_quarter"]   = df["date"].dt.to_period("Q").apply(lambda x: f"{x.year}{x.quarter:02}")
df["dt_month"]     = df["date"].dt.to_period("M").apply(lambda x: f"{x.year}{x.month:02}")
df["dt_fortnight"] = df["date"].apply(lambda x: f"{x.year}{((x.dayofyear - 1) // 14) + 1:02}")
df["dt_week"]      = df["date"].dt.strftime('%Y%U')
df["dt_day"]       = df["date"].dt.strftime("%Y%m%d")

df["is_year_start_trade"]      = (df["dt_year"] != df["dt_year"].shift(1)).astype(int)
df["is_year_end_trade"]        = (df["dt_year"] != df["dt_year"].shift(-1)).astype(int)

df["is_quarter_start_trade"]   = (df["dt_quarter"] != df["dt_quarter"].shift(1)).astype(int)
df["is_quarter_end_trade"]     = (df["dt_quarter"] != df["dt_quarter"].shift(-1)).astype(int)

df["is_month_start_trade"]     = (df["dt_month"] != df["dt_month"].shift(1)).astype(int)
df["is_month_end_trade"]       = (df["dt_month"] != df["dt_month"].shift(-1)).astype(int)

df["is_fortnight_start_trade"] = (df["dt_fortnight"] != df["dt_fortnight"].shift(1)).astype(int)
df["is_fortnight_end_trade"]   = (df["dt_fortnight"] != df["dt_fortnight"].shift(-1)).astype(int)

df["is_week_start_trade"]      = (df["dt_week"] != df["dt_week"].shift(1)).astype(int)
df["is_week_end_trade"]        = (df["dt_week"] != df["dt_week"].shift(-1)).astype(int)

df["is_year_start_order"]      = df["is_year_start_trade"].shift(-1, fill_value=0)
df["is_year_end_order"]        = df["is_year_end_trade"].shift(-1, fill_value=0)

df["is_quarter_start_order"]   = df["is_quarter_start_trade"].shift(-1, fill_value=0)
df["is_quarter_end_order"]     = df["is_quarter_end_trade"].shift(-1, fill_value=0)

df["is_month_start_order"]     = df["is_month_start_trade"].shift(-1, fill_value=0)
df["is_month_end_order"]       = df["is_month_end_trade"].shift(-1, fill_value=0)

df["is_fortnight_start_order"] = df["is_fortnight_start_trade"].shift(-1, fill_value=0)
df["is_fortnight_end_order"]   = df["is_fortnight_end_trade"].shift(-1, fill_value=0)

df["is_week_start_order"]      = df["is_week_start_trade"].shift(-1, fill_value=0)
df["is_week_end_order"]        = df["is_week_end_trade"].shift(-1, fill_value=0)

df['day_of_quarter_nature_trade']   = df['date'].apply(lambda x: (x - x.to_period('Q').start_time).days + 1)
df['day_of_year_nature_trade']      = df['date'].apply(lambda x: (x - x.to_period('Y').start_time).days + 1)
df['day_of_month_nature_trade']     = df['date'].apply(lambda x: x.day)
df['day_of_fortnight_nature_trade'] = df['date'].apply(lambda x: ((x.dayofyear - 1) % 14) + 1)
df['day_of_week_nature_trade']      = df['date'].dt.dayofweek + 1 

df['day_of_quarter_nature_order']   = df['day_of_quarter_nature_trade'].shift(-1, fill_value=0)
df['day_of_year_nature_order']      = df['day_of_year_nature_trade'].shift(-1, fill_value=0)
df['day_of_month_nature_order']     = df['day_of_month_nature_trade'].shift(-1, fill_value=0)
df['day_of_fortnight_nature_order'] = df['day_of_fortnight_nature_trade'].shift(-1, fill_value=0)

df["day_of_year_trading_trade"]      = df.groupby("dt_year").cumcount() + 1
df["day_of_quarter_trading_trade"]   = df.groupby("dt_quarter").cumcount() + 1
df["day_of_month_trading_trade"]     = df.groupby("dt_month").cumcount() + 1
df["day_of_fortnight_trading_trade"] = df.groupby("dt_fortnight").cumcount() + 1
df["day_of_week_trading_trade"]      = df.groupby("dt_week").cumcount() + 1

df["day_of_year_trading_order"]      = df["day_of_year_trading_trade"].shift(-1, fill_value=0)
df["day_of_quarter_trading_order"]   = df["day_of_quarter_trading_trade"].shift(-1, fill_value=0)
df["day_of_month_trading_order"]     = df["day_of_month_trading_trade"].shift(-1, fill_value=0)
df["day_of_fortnight_trading_order"] = df["day_of_fortnight_trading_trade"].shift(-1, fill_value=0)
df["day_of_week_trading_order"]      = df["day_of_week_trading_trade"].shift(-1, fill_value=0)

In [4]:
df

,date,date_index,dt_year,dt_quarter,dt_month,dt_fortnight,dt_week,dt_day,is_year_start_trade,is_year_end_trade,...,day_of_year_trading_trade,day_of_quarter_trading_trade,day_of_month_trading_trade,day_of_fortnight_trading_trade,day_of_week_trading_trade,day_of_year_trading_order,day_of_quarter_trading_order,day_of_month_trading_order,day_of_fortnight_trading_order,day_of_week_trading_order
0,1990-12-03,1,1990,199004,199012,199025,199048,19901203,1,0,...,1,1,1,1,1,2,2,2,2,2
1,1990-12-04,2,1990,199004,199012,199025,199048,19901204,0,0,...,2,2,2,2,2,3,3,3,3,3
2,1990-12-05,3,1990,199004,199012,199025,199048,19901205,0,0,...,3,3,3,3,3,4,4,4,4,4
3,1990-12-06,4,1990,199004,199012,199025,199048,19901206,0,0,...,4,4,4,4,4,5,5,5,5,5
4,1990-12-07,5,1990,199004,199012,199025,199048,19901207,0,0,...,5,5,5,5,5,6,6,6,6,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8562,2025-12-25,8563,2025,202504,202512,202526,202551,20251225,0,0,...,239,56,19,7,4,240,57,20,8,5
8563,2025-12-26,8564,2025,202504,202512,202526,202551,20251226,0,0,...,240,57,20,8,5,241,58,21,9,1
8564,2025-12-29,8565,2025,202504,202512,202526,202552,20251229,0,0,...,241,58,21,9,1,242,59,22,10,2
8565,2025-12-30,8566,2025,202504,202512,202526,202552,20251230,0,0,...,242,59,22,10,2,243,60,23,1,3


In [5]:
# Save to DAI
ds = dai.DataSource("mldt_cn_trading_calendar")
def update_df(df_new):
    df_new = df
    return df_new
ds.apply_bdb(update_df, as_type=pd.DataFrame)

dai.DataSource.write_bdb(
    data=df,
    id="mldt_cn_trading_calendar",
    unique_together=["date"],
    indexes=["date"],
)

dai.DataSource("mldt_cn_trading_calendar")